<a href="https://colab.research.google.com/github/simlexx-k/QueenBeeAcoustics/blob/main/Queen_Bee_detection_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
harshkumar1711_beehive_audio_dataset_with_queen_and_without_queen_path = kagglehub.dataset_download('harshkumar1711/beehive-audio-dataset-with-queen-and-without-queen')

print('Data source import complete.')


100%|██████████| 4.15G/4.15G [00:37<00:00, 118MB/s]

Extracting files...


Data source import complete.


In [8]:
print(harshkumar1711_beehive_audio_dataset_with_queen_and_without_queen_path)

/root/.cache/kagglehub/datasets/harshkumar1711/beehive-audio-dataset-with-queen-and-without-queen/versions/7


In [9]:
!cp -r "$harshkumar1711_beehive_audio_dataset_with_queen_and_without_queen_path" /content/beehive_audio

# Machine Learning Pipeline for Queen Bee Acoustic Monitoring Using CNN

A complete end-to-end prototype to detect **Queen Bee Presence** from **Hive audio recordings** using a Machine Learning pipeline.

---

## Live Demo (CNN prototype is in progress)
<div class="alert alert-block alert-success">
Note:
You can try the deployed model (Model used Random Forest Classifier in this prototype) and download a sample audio file to test it.

</div>

<span style="display: inline-block;">
  <a href="http://3.109.237.216">
    <img src="https://img.shields.io/badge/Live-Demo-blue" alt="Live Demo">
  </a>
</span>

<span style="display: inline-block;" >
<a href="https://drive.google.com/drive/folders/1KFmHH304soDKXbLvlohWBJ-qVzudjTgZ?usp=sharing">
    <img src="https://img.shields.io/badge/Google%20Drive-Sample%20Data-blue?logo=google-drive&logoColor=white" alt="Google Drive">
</a>
</span>
    
<span style="display: inline-block;">
<a href="https://github.com/Harsh-1711/BuzzDetect">
  <img src="https://img.shields.io/badge/GitHub-View%20Repository-blue?logo=github" alt="GitHub Repo">
</a>
</span>
<span style="display: inline-block;">
  <a href="https://drive.google.com/file/d/1XldGWp5G2Ecby9GLpalsgegvOgMsL_HJ/view?usp=sharing" target="_blank">
    <img src="https://img.shields.io/badge/Demo-Video-green?logo=google-drive" alt="Demo Video">
  </a>
</span>

---

This notebook demonstrates a complete pipeline for processing hive audio recordings and applying machine learning to detect the presence of the queen bee.
It covers audio feature extraction, dataset preparation, model training, evaluation, and inference.

In [3]:
"""
Import necessary libraries for the Queen Bee Detection project.

- os: For file and directory operations.
- numpy: For numerical operations on arrays and matrices.
- matplotlib & seaborn: For data visualization.
- librosa: For audio processing and feature extraction.
"""
!pip install keras-tuner tensorflow
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import keras_tuner as kt

import warnings
warnings.filterwarnings("ignore")

## **1. Data Loading and Exploration**
#### 📌 Objective:
    Load the dataset dataset and plot a graph showing the total no. of dataset for each category.
    
<div class="alert alert-block alert-success">
    Dataset Source:
</div>
        
  - This dataset was originally sourced from https://zenodo.org/records/2667806.
  - It contains labeled audio recordings from beehives for Queen Bee detection.
  - Modified Dataset - https://www.kaggle.com/datasets/harshkumar1711/beehive-audio-dataset-with-queen-and-without-queen

In [17]:
#importing dataset from kaggle

for dirname, _, filenames in os.walk('/content/beehive_audio'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/beehive_audio/Dataset/queenbee_final_model.h5
/content/beehive_audio/Dataset/External Noise/elephant_1_part_29.wav
/content/beehive_audio/Dataset/External Noise/traffic_1_part_60.wav
/content/beehive_audio/Dataset/External Noise/horse_1_part_2.wav
/content/beehive_audio/Dataset/External Noise/horse_2_part_4.wav
/content/beehive_audio/Dataset/External Noise/rainfall_1_part_45.wav
/content/beehive_audio/Dataset/External Noise/car_68.wav
/content/beehive_audio/Dataset/External Noise/cat_1_part_29.wav
/content/beehive_audio/Dataset/External Noise/bike_13.wav
/content/beehive_audio/Dataset/External Noise/wind_1_part_188.wav
/content/beehive_audio/Dataset/External Noise/car_89.wav
/content/beehive_audio/Dataset/External Noise/truck_1.wav
/content/beehive_audio/Dataset/External Noise/dog_2_part_39.wav
/content/beehive_audio/Dataset/External Noise/rainfall_1_part_101.wav
/content/beehive_audio/Dataset/External Noise/horse_2_part_47.wav
/content/beehive_audio/Dataset/External Noise/hor

In [4]:

# Load base path
base_path = "/content/beehive_audio/Dataset/Bee Hive Audios"

# Include spaces prbboperly using raw string or quotes
present_path = os.path.join(base_path, "QueenBee Present")
absent_path = os.path.join(base_path, "QueenBee Absent")
external_path= '/content/beehive_audio/Dataset/External Noise'

# Count .wav files
present_count = len([f for f in os.listdir(present_path) if f.endswith('.wav')])
absent_count = len([f for f in os.listdir(absent_path) if f.endswith('.wav')])
external_count = len([f for f in os.listdir(external_path) if f.endswith('.wav')])
# Print counts
print(f"QueenBee Present: {present_count} files")
print(f"QueenBee Absent: {absent_count} files")
print(f"External Noise: {external_count} files")

# Plot the counts
labels = ['QueenBee Present', 'QueenBee Absent','External Noise']
counts = [present_count, absent_count, external_count]

plt.figure(figsize=(6, 4))
plt.ylim(0, 4300)
bars=plt.bar(labels, counts, color=['brown', 'beige','orange'],edgecolor='black')
plt.grid(axis="y",linestyle="--",alpha=0.7)
for bar in bars:
    x = bar.get_x() + bar.get_width()/2
    y = bar.get_height()
    plt.text(x, y+y ** 0.5, str(y), ha='center', va='bottom', fontsize=10, color='black')
plt.title("Queen Bee Audio File Count")
plt.ylabel("Number of Files")
plt.show()


QueenBee Present: 4000 files
QueenBee Absent: 2000 files
External Noise: 2000 files


## **2. Audio Preprocessing and Spectrogram Generation**
#### 📌 Objective:
    This section defines parameters for audio sampling and a function to convert `.wav` audio files into mel spectrogram images. The function performs the following steps:
    - Loads and trims the audio.
    - Normalizes the signal and ensures a fixed length.
    - Generates a mel spectrogram using Librosa.
    - Saves the spectrogram as a `.png` image for use in model training.

This is a key step in preparing audio data for convolutional neural networks (CNNs), which work better with visual inputs.

In [5]:
output_base = "/content/beehive_audio/Dataset/Spectograms"
os.makedirs(os.path.join(output_base, "present"), exist_ok=True)
os.makedirs(os.path.join(output_base, "absent"), exist_ok=True)
os.makedirs(os.path.join(output_base, "external"), exist_ok=True)


In [6]:
SAMPLE_RATE = 22050
DURATION = 3  # seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

# Function to process and save spectrograms
def preprocess_and_save_spectrogram(audio_path, output_image_path, sr=SAMPLE_RATE, duration=DURATION):

    """
    Load an audio file, convert it into a mel spectrogram, and save it as an image.

    Parameters:
    -----------
    audio_path : str
        Path to the input .wav audio file.
    output_image_path : str
        Path where the output spectrogram image will be saved.
    sr : int, optional
        Sampling rate for the audio. Default is 22050 Hz.
    duration : int, optional
        Duration (in seconds) to which the audio will be fixed. Default is 3 seconds.

    Steps:
    ------
    1. Load and trim silence from the audio.
    2. Convert to mono and normalize the waveform.
    3. Pad or truncate to fixed duration.
    4. Generate a mel spectrogram.
    5. Convert the power spectrogram to decibels.
    6. Save the spectrogram as a `.png` image (suitable for CNN input).

    Exceptions:
    -----------
    If any error occurs during processing, the function will print an error message.
    """

    try:
        y, _ = librosa.load(audio_path, sr=sr)
        y, _ = librosa.effects.trim(y)
        y = librosa.to_mono(y) if y.ndim > 1 else y
        y = librosa.util.normalize(y)

        max_len = sr * duration
        if len(y) > max_len:
            y = y[:max_len]
        else:
            y = np.pad(y, (0, max_len - len(y)))

        # Create mel spectrogram
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        S_dB = librosa.power_to_db(S, ref=np.max)

        # Save as image
        plt.figure(figsize=(2.56, 2.56), dpi=100)
        librosa.display.specshow(S_dB, sr=sr, cmap='magma')
        plt.axis('off')
        plt.tight_layout(pad=0)
        plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0)
        plt.close()
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")


In [ ]:
for filename in tqdm(os.listdir(present_path), desc="Processing Present"):
    if filename.endswith('.wav'):
        input_path = os.path.join(present_path, filename)
        output_path = os.path.join(output_base, "present", filename.replace(".wav", ".png"))
        preprocess_and_save_spectrogram(input_path, output_path)

# Process Absent Files
for filename in tqdm(os.listdir(absent_path), desc="Processing Absent"):
    if filename.endswith('.wav'):
        input_path = os.path.join(absent_path, filename)
        output_path = os.path.join(output_base, "absent", filename.replace(".wav", ".png"))
        preprocess_and_save_spectrogram(input_path, output_path)

# Process External Noise Files
for root, dirs, files in os.walk(external_path):
    for filename in files:
        if filename.endswith('.wav'):
            input_path = os.path.join(root, filename)
            output_path = os.path.join(output_base, "external", filename.replace(".wav", ".png"))
            preprocess_and_save_spectrogram(input_path, output_path)


print("✅ Spectrogram image generation complete.")


Processing Absent: 100%|██████████| 2000/2000 [04:12<00:00,  7.91it/s]


Attempting to fix the memory climb on processing Spectograms

In [7]:
import gc
import librosa
import matplotlib.pyplot as plt

# Clear librosa cache (safe on all versions)
librosa.cache.clear()

# Non-interactive backend (prevents memory growth)
plt.switch_backend("Agg")

# Process Present Files
for filename in tqdm(os.listdir(present_path), desc="Processing Present"):
    if filename.endswith('.wav'):
        input_path = os.path.join(present_path, filename)
        output_path = os.path.join(output_base, "present", filename.replace(".wav", ".png"))

        preprocess_and_save_spectrogram(input_path, output_path)
        gc.collect()

# Process Absent Files
for filename in tqdm(os.listdir(absent_path), desc="Processing Absent"):
    if filename.endswith('.wav'):
        input_path = os.path.join(absent_path, filename)
        output_path = os.path.join(output_base, "absent", filename.replace(".wav", ".png"))

        preprocess_and_save_spectrogram(input_path, output_path)
        gc.collect()

# Process External Noise Files
for root, dirs, files in os.walk(external_path):
    for filename in files:
        if filename.endswith('.wav'):
            input_path = os.path.join(root, filename)
            output_path = os.path.join(output_base, "external", filename.replace(".wav", ".png"))

            preprocess_and_save_spectrogram(input_path, output_path)
            gc.collect()

print("✅ Spectrogram image generation complete.")


Processing Absent: 100%|██████████| 2000/2000 [12:43<00:00,  2.62it/s]


✅ Spectrogram image generation complete.


## **3. Visualizing Spectrogram Distribution Across Classes**

This section analyzes the dataset by counting the number of spectrogram images in each class — "present" and "absent" — and visualizes the distribution using a bar chart.

    Steps performed:
    - Counts `.png` spectrogram files in each class directory.
    - Stores and prints the counts.
    - Plots a bar graph to visualize class balance.
    - Adds numeric labels on top of each bar for clarity.

This helps assess whether the dataset is balanced, which is crucial for model performance in classification tasks.


In [8]:

# Spectrogram image path
spectrogram_path = "/content/beehive_audio/Dataset/Spectograms"

# Define class folders
class_folders = ['present', 'absent']
counts = []

# Count PNG files in each folder
for label in class_folders:
    class_path = os.path.join(spectrogram_path, label)
    count = len([f for f in os.listdir(class_path) if f.endswith('.png')])
    counts.append(count)
    print(f"{label}: {count} spectrograms")

# Plotting
plt.figure(figsize=(6, 4))
bars = plt.bar(class_folders, counts, color=['darkorange', 'skyblue'], edgecolor='black')
plt.ylim(0,4400)
plt.title("Number of Spectrogram PNGs per Class")
plt.ylabel("Count")
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Add count labels on top
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval+yval ** 0.5, str(yval), ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()


present: 4000 spectrograms
absent: 4000 spectrograms


## **4. Preparing Data Loaders: Train, Validation, and Test Splits**

This section prepares the spectrogram dataset for training a CNN using TensorFlow's `ImageDataGenerator`.

- **Base Directory**: Points to the location of spectrogram images organized by class.
- **Image Preprocessing**: Rescales pixel values to the [0, 1] range.
- **Splitting Strategy**:
  - **70%** of the data is used for training.
  - The remaining **30%** is further split evenly into **15% validation** and **15% testing**.
- **Generators Created**:
  - `train_gen` for training the model.
  - `val_gen` for tuning hyperparameters and monitoring overfitting.
  - `test_gen` for final performance evaluation.

This setup ensures consistent and reproducible splits with a fixed seed.


In [9]:
# Base directory
data_dir = "/content/beehive_audio/Dataset/Spectograms"

# Params
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
SEED = 42

# 💾 Data split: 70% train, 15% val, 15% test
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.30)

# 70% Train
train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=SEED
)

# Split remaining 30% into 15% val + 15% test
datagen_val_test = ImageDataGenerator(rescale=1./255, validation_split=0.5)

val_gen = datagen_val_test.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=SEED
)

test_gen = datagen_val_test.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    seed=SEED,
    shuffle=False
)


Found 7000 images belonging to 3 classes.
Found 5000 images belonging to 3 classes.
Found 5000 images belonging to 3 classes.


In [10]:
Dataset = '/content/beehive_audio/Dataset/Spectograms'

datagen = ImageDataGenerator(rescale=1./255)
gen = datagen.flow_from_directory(Dataset, class_mode='binary')
print(gen.class_indices)


Found 10000 images belonging to 3 classes.
{'absent': 0, 'external': 1, 'present': 2}


In [23]:
print(train_gen.class_indices)
print(val_gen.class_indices)
print(test_gen.class_indices)

{'absent': 0, 'external': 1, 'present': 2}
{'absent': 0, 'external': 1, 'present': 2}
{'absent': 0, 'external': 1, 'present': 2}


In [25]:
from tensorflow.keras import layers, models

def build_model():
    model = models.Sequential([
        layers.Conv2D(32, 3, activation='relu', padding='same',
                      input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.GlobalAveragePooling2D(),

        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),

        # MUST be 3 units for 3 classes
        layers.Dense(3, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        # MUST be sparse categorical for integer labels 0/1/2
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


# 🚀 Train ONCE
model = build_model()
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

Epoch 1/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 485s 2s/step - accuracy: 0.6232 - loss: 0.8188 - val_accuracy: 0.5054 - val_loss: 1.3206
Epoch 2/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 497s 2s/step - accuracy: 0.7558 - loss: 0.4499 - val_accuracy: 0.6118 - val_loss: 1.8830
Epoch 3/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 473s 2s/step - accuracy: 0.7770 - loss: 0.3819 - val_accuracy: 0.4234 - val_loss: 1.1530
Epoch 4/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 477s 2s/step - accuracy: 0.7853 - loss: 0.3886 - val_accuracy: 0.4366 - val_loss: 1.3721
Epoch 5/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 471s 2s/step - accuracy: 0.7755 - loss: 0.3693 - val_accuracy: 0.4434 - val_loss: 2.0075
Epoch 6/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 470s 2s/step - accuracy: 0.7951 - loss: 0.3488 - val_accuracy: 0.3930 - val_loss: 1.6818
Epoch 7/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 472s 2s/step - accuracy: 0.7825 - loss: 0.3430 - val_accuracy: 0.7964 - val_loss: 0.3637
Epoch 8/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 485s 2s/step - accuracy: 0.7826 - loss: 0.3497 - val_accu

In [ ]:
import keras_tuner as kt
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping

def build_model(hp):
    model = models.Sequential([
        layers.Conv2D(
            hp.Choice("conv1_filters", [32, 64]),
            3, padding="same", activation="relu",
            input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
        ),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(
            hp.Choice("conv2_filters", [64, 128]),
            3, padding="same", activation="relu"
        ),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(
            hp.Choice("conv3_filters", [128, 256]),
            3, padding="same", activation="relu"
        ),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.GlobalAveragePooling2D(),

        layers.Dense(
            hp.Int("dense_units", min_value=64, max_value=128, step=32),
            activation="relu"
        ),
        layers.Dropout(hp.Float("dropout", 0.3, 0.6, step=0.1)),

        layers.Dense(3, activation="softmax")
    ])

    lr = hp.Choice("lr", [1e-3, 5e-4, 1e-4])
    optimizer = optimizers.Adam(learning_rate=lr)

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model


tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=6,          # keep small
    overwrite=True,
    directory="queenbee_tuner",
    project_name="cnn_multiclass"
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True
)

tuner.search(
    train_gen,
    validation_data=val_gen,
    epochs=8,              # IMPORTANT: small
    callbacks=[early_stop]
)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
32                |32                |conv1_filters
128               |128               |conv2_filters
128               |128               |conv3_filters
64                |64                |dense_units
0.3               |0.3               |dropout
0.0005            |0.0005            |lr

Epoch 1/8
219/219 ━━━━━━━━━━━━━━━━━━━━ 695s 3s/step - accuracy: 0.6818 - loss: 0.7177 - val_accuracy: 0.5040 - val_loss: 1.2383
Epoch 2/8
158/219 ━━━━━━━━━━━━━━━━━━━━ 2:44 3s/step - accuracy: 0.7621 - loss: 0.4316

In [ ]:
# Combine train + val
datagen_full = ImageDataGenerator(rescale=1./255, validation_split=0.15)
final_train_gen = datagen_full.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=SEED
)
final_val_gen = datagen_full.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    seed=SEED
)

# Train
history = best_model.fit(
    final_train_gen,
    validation_data=final_val_gen,
    epochs=10
)


In [ ]:
# Save the fine-tuned model
best_model.save("/kaggle/working/queenbee_final_model.h5")

In [12]:
#load the fine-tuned model
from tensorflow.keras.models import load_model

model = load_model("/content/beehive_audio/Dataset/queenbee_final_model.h5") # change the path if you are using your own trained model


## **5.** **Evaluate Metrics**

* Accuracy, Precision, Recall, F1, ROC AUC  
* Confusion Matrix  
* ROC & PR Curves  
* Test Set Performance Summary

In [13]:
def run_inference(model, test_gen):
    """
    Runs inference on the given test data generator using a trained model.

    Args:
        model (tf.keras.Model): Trained Keras model for binary classification.
        test_gen (ImageDataGenerator): Test data generator with images and labels.

    Returns:
        tuple:
            y_pred (np.ndarray): Binary class predictions (0 or 1).
            y_prob (np.ndarray): Predicted probabilities for the positive class.
            y_true (np.ndarray): Ground truth labels from the test generator.
    """
    test_gen.reset()
    y_prob = model.predict(test_gen, steps=int(np.ceil(test_gen.samples / test_gen.batch_size))).ravel()
    y_pred = (y_prob > 0.5).astype(int)

    return y_pred, y_prob, test_gen.classes

In [14]:
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix,
    classification_report, roc_curve, precision_recall_curve
)
import matplotlib.pyplot as plt
import seaborn as sns

# 📈 Metrics
y_pred, y_prob, y_true = run_inference(model, test_gen)
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_prob)
pr_auc = average_precision_score(y_true, y_prob)

print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"AU-ROC:    {roc_auc:.4f}")
print(f"AU-PRC:    {pr_auc:.4f}")


157/157 ━━━━━━━━━━━━━━━━━━━━ 52s 330ms/step


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

Added average declarations for multiclass **targettig**

In [17]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score
)

y_pred, y_prob, y_true = run_inference(model, test_gen)

acc = accuracy_score(y_true, y_pred)

prec = precision_score(y_true, y_pred, average="macro")
rec  = recall_score(y_true, y_pred, average="macro")
f1   = f1_score(y_true, y_pred, average="macro")

# ROC / PR only make sense for binary scores
roc_auc = roc_auc_score(y_true == 1, y_prob)
pr_auc  = average_precision_score(y_true == 1, y_prob)

print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"AU-ROC:    {roc_auc:.4f}")
print(f"AU-PRC:    {pr_auc:.4f}")

157/157 ━━━━━━━━━━━━━━━━━━━━ 62s 390ms/step
Accuracy:  0.3896
Precision: 0.2203
Recall:    0.3247
F1 Score:  0.2625
AU-ROC:    0.0476
AU-PRC:    0.1087


In [18]:
# 📊 Confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=test_gen.class_indices.keys(), yticklabels=test_gen.class_indices.keys())
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

#ROC curve

fpr, tpr, _ = roc_curve(y_true, y_prob)

plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f"AU-ROC = {roc_auc:.4f}")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid()
plt.show()

#precision recall curve

precision, recall, _ = precision_recall_curve(y_true, y_prob)

plt.figure(figsize=(6, 4))
plt.plot(recall, precision, label=f"AU-PRC = {pr_auc:.4f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.grid()
plt.show()

#Histogram of the predicted model

plt.figure(figsize=(6, 4))
plt.hist(y_prob[y_true == 0], bins=25, alpha=0.7, label='Class 0')
plt.hist(y_prob[y_true == 1], bins=25, alpha=0.7, label='Class 1')
plt.xlabel("Predicted Probability")
plt.ylabel("Frequency")
plt.title("Predicted Probabilities by True Class")
plt.legend()
plt.grid()
plt.show()




ValueError: multiclass format is not supported

In [ ]:
print(classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys())))


### Model Evaluation Commentary
The model demonstrates excellent performance with an **overall accuracy of 99%**. Both classes — "absent" and "present" — have near-perfect **precision, recall, and F1-scores (0.99)**, indicating:

**High precision**: Very few false positives.

**High recall**: Nearly all relevant instances were correctly identified.

**Balanced performance**: Consistent metrics across both classes, confirmed by the macro and weighted averages.

## **6. Prediction and Audio Feature Visualization**
### 📌 Objective:
Visualize key audio features from a sound signal to better understand the characteristics of the data.

#### Features Plotted:
    - Raw audio waveform
    - Mel-spectrogram input to CNN
    - Model prediction probabilities (Queen Present/Absent)
    - Audio statistics (duration, RMS energy, zero-crossing rate, etc.)

In [19]:
def visualize_audio_prediction(audio_path, model):
    """
    Create essential visualizations for audio CNN prediction
    """
    # Load audio
    y, sr = librosa.load(audio_path, sr=SR)

    # Create mel-spectrogram
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    S_dB = librosa.power_to_db(S, ref=np.max)

    # Get model input (spectrogram image)
    input_data = audio_to_spectrogram_image(audio_path, model)

    # Get prediction
    prediction = model.predict(input_data)
    queen_prob = float(prediction[0][0])
    label = "Queen Present" if queen_prob >= 0.5 else "Queen Absent"
    confidence = queen_prob if queen_prob >= 0.5 else (1 - queen_prob)

    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Audio Analysis: {audio_path.split("/")[-1]}', fontsize=16, fontweight='bold')

    # 1. Raw Waveform
    axes[0, 0].plot(np.linspace(0, len(y)/sr, len(y)), y, color='blue', alpha=0.7)
    axes[0, 0].set_title('Raw Audio Waveform', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Time (seconds)')
    axes[0, 0].set_ylabel('Amplitude')
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Mel-Spectrogram (what CNN sees)
    im = axes[0, 1].imshow(S_dB, aspect='auto', origin='lower', cmap='magma')
    axes[0, 1].set_title('Mel-Spectrogram (CNN Input)', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Time Frames')
    axes[0, 1].set_ylabel('Mel Frequency Bins')
    plt.colorbar(im, ax=axes[0, 1], label='Power (dB)')

    # 3. Prediction Results
    categories = ['Queen Absent', 'Queen Present']
    probabilities = [1 - queen_prob, queen_prob]
    colors = ['red' if p == max(probabilities) else 'lightcoral' for p in probabilities]

    bars = axes[1, 0].bar(categories, probabilities, color=colors, alpha=0.8, edgecolor='black')
    axes[1, 0].set_title('Model Predictions', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('Probability')
    axes[1, 0].set_ylim(0, 1)
    axes[1, 0].grid(True, alpha=0.3, axis='y')

    # Add probability labels on bars
    for bar, prob in zip(bars, probabilities):
        height = bar.get_height()
        axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{prob:.3f}', ha='center', va='bottom', fontweight='bold')

    # 4. Audio Statistics
    axes[1, 1].axis('off')
    duration = len(y) / sr
    max_amplitude = np.max(np.abs(y))
    rms_energy = np.sqrt(np.mean(y**2))
    zero_crossings = np.sum(np.diff(np.sign(y)) != 0) / len(y)

    stats_text = f"""
    Audio Statistics:

    Duration: {duration:.2f} seconds
    Sample Rate: {sr} Hz
    Max Amplitude: {max_amplitude:.4f}
    RMS Energy: {rms_energy:.4f}
    Zero Crossing Rate: {zero_crossings:.4f}

    Prediction Results:

    Final Prediction: {label}
    Confidence: {confidence*100:.2f}%

    Queen Present Prob: {queen_prob:.3f}
    Queen Absent Prob: {1-queen_prob:.3f}
    """

    axes[1, 1].text(0.1, 0.9, stats_text, transform=axes[1, 1].transAxes,
                   fontsize=12, verticalalignment='top',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray", alpha=0.8))

    plt.tight_layout()
    plt.show()

    return {
        'prediction': label,
        'confidence': confidence,
        'queen_probability': queen_prob,
        'duration': duration,
        'audio_stats': {
            'max_amplitude': max_amplitude,
            'rms_energy': rms_energy,
            'zero_crossing_rate': zero_crossings
        }
    }

In [21]:
from PIL import Image
import io

# --- Parameters (must match training setup) ---
IMG_SIZE = (128, 128)  # your model's input size
SR = 22050  # sampling rate used during training

# --- Function to process a .wav file into spectrogram image ---
def audio_to_spectrogram_image(audio_path,model=None):
    y, sr = librosa.load(audio_path, sr=SR)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    S_dB = librosa.power_to_db(S, ref=np.max)

    # Plot to image in memory
    fig = plt.figure(figsize=(2, 2), dpi=64)  # ~128x128
    librosa.display.specshow(S_dB, sr=sr, cmap='magma')
    plt.axis('off')
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    buf.seek(0)

    # Load image and resize
    img = Image.open(buf).convert('RGB').resize(IMG_SIZE)
    img_array = np.array(img) / 255.0  # Normalize like training
    img_array = np.expand_dims(img_array, axis=0)  # Shape: (1, 128, 128, 3)
    return img_array

# --- Predict ---
audio_file_path = "/content/beehive_audio/Dataset/Bee Hive Audios/QueenBee Absent/Hive1 31_05_2018_NO_QueenBee____00_00_00_chunk1.wav"
# run_inference(model,test_gen)
input_data = audio_to_spectrogram_image(audio_file_path)

results = visualize_audio_prediction(audio_file_path, model)

print(f"\n{'='*50}")
print("PREDICTION SUMMARY")
print(f"{'='*50}")
print(f"File: {audio_file_path.split('/')[-1]}")
print(f"Prediction: {results['prediction']}")
print(f"Confidence: {results['confidence']*100:.2f}%")
print(f"Audio Duration: {results['duration']:.2f} seconds")
print(f"{'='*50}")

audio_file_path = "/content/beehive_audio/Dataset/Bee Hive Audios/QueenBee Present/Hive1 12_06_2018_QueenBee____00_00_00_chunk1.wav"
# run_inference(model,test_gen)
input_data = audio_to_spectrogram_image(audio_file_path)

results = visualize_audio_prediction(audio_file_path, model)

print(f"\n{'='*50}")
print("PREDICTION SUMMARY")
print(f"{'='*50}")
print(f"File: {audio_file_path.split('/')[-1]}")
print(f"Prediction: {results['prediction']}")
print(f"Confidence: {results['confidence']*100:.2f}%")
print(f"Audio Duration: {results['duration']:.2f} seconds")
print(f"{'='*50}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step

PREDICTION SUMMARY
File: Hive1 31_05_2018_NO_QueenBee____00_00_00_chunk1.wav
Prediction: Queen Absent
Confidence: 94.89%
Audio Duration: 10.00 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step

PREDICTION SUMMARY
File: Hive1 12_06_2018_QueenBee____00_00_00_chunk1.wav
Prediction: Queen Present
Confidence: 100.00%
Audio Duration: 10.00 seconds
